<a href="https://colab.research.google.com/github/ducviet00/Generating-TruyenKieu-Style/blob/master/TruyenKieu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install underthesea

     |████████████████████████████████| 7.1MB 5.9MB/s 
     |████████████████████████████████| 460kB 53.2MB/s 
     |████████████████████████████████| 757kB 50.8MB/s 
     |████████████████████████████████| 1.5MB 54.3MB/s 
     |████████████████████████████████| 5.4MB 22.0MB/s 
     |████████████████████████████████| 286kB 58.4MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449907 sha256=e1f5ae50865221b5664799e8e1c8af76e5ed1dc6771f47a61c37ede592f9ecfd
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for clint: filename=clint-0.5.1-cp36-none-any.whl size=34449 sha256=c8d1c3aeaf429b9fd7b26d84505be0d606301dd9899b02a88f33ec66d33c5504
  Stored in directory: /root/.cache/pip/wheels/4f/e9/45/223565e5b1a4b09e12c6de6f8ba7c2c0e9127dec17cf830f83
  Created wheel for args: filename=args-0.1.0-cp36-none-any.whl size=3320 sha256=6a4acf210086ecb3edcdc79c924606b4475955d7573b878fafdd75fda9321793
  Store

In [ ]:
import tensorflow as tf
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import utils

Using TensorFlow backend.


In [ ]:
from keras.optimizers import Adam
from keras.optimizers import RMSprop

In [ ]:
from underthesea import word_tokenize

In [ ]:
import numpy as np
import pandas as pd
import re


In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/TruyenKieu/truyen_kieu_data.txt'
df = pd.read_csv(path, sep="/", names=["row"], encoding="utf8").dropna()
df.head(10)

,row
0,"1..Trăm năm trong cõi người ta,"
1,2..Chữ tài chữ mệnh khéo là ghét nhau.
2,"3..Trải qua một cuộc bể dâu,"
3,4..Những điều trông thấy mà đau đớn lòng.
4,"5.. Lạ gì bỉ sắc tư phong,"
5,6..Trời xanh quen thói má hồng đánh ghen.
6,"7..Cảo thơm lần giở trước đèn,"
7,8..Phong tình có lục còn truyền sử xanh.
8,"9,,Rằng năm Gia Tĩnh triều Minh,"
9,"10.. Bốn phương phẳng lặng, hai kinh vững vàng."


In [ ]:
def transfrom(row):
    s = []
    punctuation = ['.', ',', '\"', '\'', '?', ':', '!', '-', ';']
    for i in row:
        i = i.lower()
        i = re.sub(r"^[0-9\., ]+", "", i)
        i = re.sub(r"[\.,\?]+$", "", i)
        for p in punctuation:
            i = i.replace(p, ' ')
        s.append(i)
    return s

In [ ]:
corpus = transfrom(df["row"])

In [ ]:
corpus[:20]

['trăm năm trong cõi người ta',
 'chữ tài chữ mệnh khéo là ghét nhau',
 'trải qua một cuộc bể dâu',
 'những điều trông thấy mà đau đớn lòng',
 'lạ gì bỉ sắc tư phong',
 'trời xanh quen thói má hồng đánh ghen',
 'cảo thơm lần giở trước đèn',
 'phong tình có lục còn truyền sử xanh',
 'rằng năm gia tĩnh triều minh',
 'bốn phương phẳng lặng  hai kinh vững vàng',
 'có nhà viên ngoại họ vương',
 'gia tư nghĩ cũng thường thường bực trung',
 'một trai con thứ rốt lòng',
 'vương quan là chữ  nối dòng nho gia',
 'đầu lòng hai ả tố nga',
 'thúy kiều là chị  em là thúy vân',
 'mai cốt cách  tuyết tinh thần',
 'một người một vẻ  mười phân vẹn mười',
 'vân xem trang trọng khác vời',
 'khuôn trăng đầy đặn  nét ngài nở nang']

In [ ]:
i = word_tokenize(corpus[9])
print(i)

['bốn phương', 'phẳng lặng', 'hai', 'kinh', 'vững vàng']


In [ ]:
def word_seg(corpus):
    c = []
    for i in corpus:
        sentence = word_tokenize(i)
        for j in sentence:
            c.append(j)
        c.append('\n')
    return c

In [ ]:
context = word_seg(corpus=corpus)

In [ ]:
len(context)

22703

In [ ]:
import collections

def word2index(context):
    word_counter = collections.Counter(context).most_common()

    word2idx = dict()
    idx2word = dict()

    for word,i in word_counter:
        index = len(word2idx)
        word2idx[word] = index
        idx2word[index] = word
    
    vocab_size = len(word2idx)

    return word2idx, idx2word, vocab_size

In [ ]:
word2idx, idx2word, vocab_size = word2index(context)

In [ ]:
w2i_seq = [word2idx[word] for word in context]

In [ ]:
print('The first ten words: ', context[:10])
print('The corresponding int sequence: ', w2i_seq[:10])

The first ten words:  ['trăm', 'năm', 'trong', 'cõi', 'người ta', '\n', 'chữ', 'tài', 'chữ', 'mệnh']
The corresponding int sequence:  [103, 62, 18, 471, 472, 0, 125, 185, 125, 1192]


In [ ]:
print(vocab_size)

3922


In [ ]:
seq_length = 29

x = list()
y = list()

for i in range(seq_length, len(w2i_seq)):
    seq = w2i_seq[i-seq_length: i]
    x.append(seq)
    y.append(w2i_seq[i])

x = np.array(x)

y = utils.to_categorical(y, num_classes=vocab_size)

print('Shape of inputSequences: ', x.shape)
print('Shape of outputs: ', y.shape)

Shape of inputSequences:  (22680, 29)
Shape of outputs:  (22680, 3922)


In [ ]:
embedding_dim=300
lstm1_unit=64
lstm2_unit=128
hidden=128

model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))

model.add(LSTM(units=lstm1_unit, return_sequences=True))

model.add(LSTM(units=lstm2_unit))

model.add(Dense(hidden, activation='relu'))

model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 300)           1176600   
_________________________________________________________________
lstm_3 (LSTM)                (None, 29, 64)            93440     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 3922)              505938    
Total params: 1,891,306
Trainable params: 1,891,306
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=RMSprop(lr=0.001, clipvalue=0.5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=200)

In [ ]:
model.save("/content/drive/My Drive/Colab Notebooks/TruyenKieu/model.h5")

In [ ]:
del model
from keras.models import load_model
model = load_model("/content/drive/My Drive/Colab Notebooks/TruyenKieu/model.h5")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
input_test = x[0]

In [ ]:
predicted_words = [idx2word[index] for index in input_test]

In [ ]:
for i in range(50):
    pred = model.predict_classes(input_test.reshape((1,seq_length)), verbose=0)

    input_test = np.append(input_test, pred[0])
    input_test = input_test[1:]

    predicted_words.append(idx2word[pred[0]])


In [ ]:
print('\nThe words sequence generated by the Model: ')
print(' '.join(predicted_words))


The words sequence generated by the Model: 
trăm năm trong cõi người ta 
 chữ tài chữ mệnh khéo là ghét nhau 
 trải qua một cuộc bể dâu 
 những điều trông thấy mà đau đớn lòng 
 lạ gì bỉ sắc tư phong 
 trời xanh quen thói má hồng đánh ghen 
 cảo thơm lần giở trước đèn 
 phong tình có lục còn truyền sử xanh 
 rằng năm gia tĩnh triều minh 
 bốn phương phẳng lặng hai kinh vững vàng 
 có nhà viên ngoại họ vương 
 gia tư nghĩ cũng


In [ ]:
print(' '.join(context[:100]))

trăm năm trong cõi người ta 
 chữ tài chữ mệnh khéo là ghét nhau 
 trải qua một cuộc bể dâu 
 những điều trông thấy mà đau đớn lòng 
 lạ gì bỉ sắc tư phong 
 trời xanh quen thói má hồng đánh ghen 
 cảo thơm lần giở trước đèn 
 phong tình có lục còn truyền sử xanh 
 rằng năm gia tĩnh triều minh 
 bốn phương phẳng lặng hai kinh vững vàng 
 có nhà viên ngoại họ vương 
 gia tư nghĩ cũng thường thường bực trung 
 một trai con thứ rốt lòng 
 vương quan là chữ nối dòng nho gia 
 đầu lòng hai ả tố nga
